# Proceso ETL

El proceso ETL es una consecución de acciones que se realizan en el marco de la gestión de bases de datos.

Consiste en extraer datos de diversas fuentes, transformarlos según las necesidades, y cargarlos en un repositorio para su análisis. Es crucial en data warehousing y análisis de grandes volúmenes de datos.

A continuación vamos a ver un ejemplo práctico de cómo funciona, usando la base de datos que creamos de las bandas de música, en concreto la tabla albums.

Primero vamos a importar unos ficheros csv con nueva información, **Extract**

Después vamos a hacer una transformación de algunas variables, **Transform**

Finalmente vamos a cargar la nueva información en las ya existentes bases de datos, **Load**

In [ ]:
# !pip install sqlalchemy
# !pip install sqlalchemy_utils
# !pip install pymysql
# !pip install mysql-connector

In [30]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date, Float, Enum, ForeignKey
from sqlalchemy.dialects.mysql import LONGTEXT

from sqlalchemy_utils import database_exists, create_database, drop_database

from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker

import pymysql

import pandas as pd
import datetime as datetime

In [21]:
Session = sessionmaker(bind=engine)
session = Session()

## Extracción datos

Creamos una función que importa los datos nuevos de que se disponen, en este caso un csv con los nuevos álbumes

In [ ]:
def extract(csv_file, columns):
    
    df = pd.read_csv(csv_file, sep = ';', names = columns)
    
    return df

## Transformación datos

Los fuente de datos, como ya la conocemos con anterioridad, sabemos que viene con un determinado formato. Por ejemplo, puede contener nulos, la fecha puede venir en formato extraño, y el tiempo viene en segundos.

A continuación vamos a transformar los datos de tal forma que casen con lo ya existente en la bbdd.

El objetivo es definir la función trasformación sabiendo que los datos siempre vienen de la misma forma y que siempre vamos a querer realizar las mismas transformaciones, de ese modo podemos automatizar el proceso

In [ ]:
def transform(df):
    
    # Eliminamos las líneas con algún nulo
    df = df.dropna()
    
    # Convertimos la fecha de lanzamiento a formato fecha
    try:
        df['release_date'] = pd.to_datetime(df['release_date'], infer_datetime_format = True)
    except:
        pass
    
    # Transformamos la duración del album de segundos a minutos
    df['run_time'] = df['run_time'] / 60
    
    return df

## Carga de datos

Similar al proceso realizado durante la creación y primera carga de datos, vamos a cargar los nuevos.

Gracias a la sección Transform sabemos cómo vienen los datos, que están en formato adecuado, y qué información hay en cada columna. Así que ejecutamos la siguiente función que hara un `.add()` en la tabla que le hayamos indicado, en este caso album

In [22]:
def load(df):
     
    for i, val in enumerate(df.values):
        
        if table == 'Band':
            rec = Band (
                band_id = val[0],
                band_url = val[1],
                band_name = val[2]
            )
        elif table == 'Album':
            rec = Album (
                album_id = val[0],
                band_id = val[1],
                album_name = val[2],
                release_date = val[3],
                abstract = val[4],
                run_time = val[5],
                sales_amount = val[6]
            )
        elif table == 'Band_genre':
            rec = Band_genre (
                band_id = val[0],
                genre_name = val[1]
            )
        elif table == 'Musician':
            rec = Musician (
                musician_id = val[0],
                musician_url = val[1]
            )
        elif table == 'Band_musician':
            rec = Band_musician (
                band_id = val[0],
                musician_id = val[1],
                musician_status = val[2]
            )
        else: # musician_name
            rec = Musician_name (
                musician_id = val[0],
                musician_name = val[1]
            )
        
        session.add(rec)
    session.commit()

## Ejecución del proceso completo

La función ETL reune todas las acciones del proceso ETL entero. Posteriormente la ejecutamos indicando los parámetros necesarios.

In [ ]:
def ETL(path_file, table_name, columns):
    
    df_extract = extract(path_file, columns)
    
    df_transform = transform(df_extract)
    
    load(df_transform, table_name)
    

In [24]:
ETL('./table_album_ETL.csv','Album', ['album_id','band_id','album_name','release_date','abstract','run_time','sales_amount'])